In [111]:
# Reference webiste for the model: https://towardsdatascience.com/time-series-forecasting-with-recurrent-neural-networks-74674e289816

#Imports
import pandas as pd
import numpy as np
import keras
import tensorflow as tf
from keras.preprocessing.sequence import TimeseriesGenerator
import numpy as np
import pandas as pd
from numpy.random import seed
seed(1)
from tensorflow import random
random.set_seed(2)
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from pathlib import Path
%matplotlib inline
import plotly.graph_objs as go

In [112]:
#Importing CSV
csvpath = Path("Resources/VNQ.csv")
ETF = pd.read_csv(csvpath)
ETF.head()


,Date,open,high,low,close,volume
0,2011-05-02 00:00:00-04:00,62.13,62.36,61.40,61.90,2324829
1,2011-05-03 00:00:00-04:00,61.74,62.19,60.69,61.39,1828226
2,2011-05-04 00:00:00-04:00,61.29,61.49,60.75,61.05,1577434
3,2011-05-05 00:00:00-04:00,60.71,61.45,60.52,60.97,1846927
4,2011-05-06 00:00:00-04:00,61.50,61.55,60.29,60.45,2121920


In [113]:
# Removing all columns but close_price
ETF.drop(columns=['open', 'high', 'low', 'volume'], inplace=True)
ETF.head()

,Date,close
0,2011-05-02 00:00:00-04:00,61.90
1,2011-05-03 00:00:00-04:00,61.39
2,2011-05-04 00:00:00-04:00,61.05
3,2011-05-05 00:00:00-04:00,60.97
4,2011-05-06 00:00:00-04:00,60.45


In [114]:
#Checking For Nulls
ETF.isnull().sum()

Date     0
close    0
dtype: int64

In [115]:
#Establishing test and train data for Closing price as well as the Date
close_data = ETF['close'].values
close_data = close_data.reshape((-1,1))

In [116]:
split_percent = 0.7
split = int(split_percent*len(close_data))

In [117]:
close_train = close_data[:split]
close_test = close_data[split:]
close_train

array([[61.9 ],
       [61.39],
       [61.05],
       ...,
       [74.21],
       [74.71],
       [74.14]])

In [118]:
date_data = ETF['Date']
date_train = ETF['Date'][:split]
date_test = ETF['Date'][split:]
print(len(close_train))
print(len(close_test))

1710
733


In [119]:
# setting the lookback, shape and 
look_back = 90

shape = (look_back,1)
# Reformat the data set to dataset of windows over a period of time.
#TimeseriesGenerator(Data, Target, sequence_length, batch)

train_generator = TimeseriesGenerator(close_train, close_train, length=look_back, batch_size=20)     
test_generator = TimeseriesGenerator(close_test, close_test, length=look_back, batch_size=1)

In [120]:
#Build LSTM Model
model = Sequential()
model.add(
    LSTM(10,
        activation='relu',
        input_shape= shape,
))
model.add(Dense(1))

# Compile the model
model.compile(optimizer='adam', loss='mse')


In [121]:
# Show the model summary
model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_7 (LSTM)                (None, 10)                480       
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 11        
Total params: 491
Trainable params: 491
Non-trainable params: 0
_________________________________________________________________


In [122]:
# Train the model
#used fit_generator because we are looking to generate new predicted data points 
num_epochs = 200
model.fit_generator(train_generator, epochs=num_epochs, verbose=1)

C:\Users\lwbak\.conda\lib\site-packages\tensorflow\python\keras\engine\training.py:1844: UserWarning:

`Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.



Epoch 1/200
81/81 [==============================] - 4s 26ms/step - loss: 20747.8443
Epoch 2/200
81/81 [==============================] - 2s 25ms/step - loss: 16783.1192
Epoch 3/200
81/81 [==============================] - 2s 25ms/step - loss: 4124.6992
Epoch 4/200
81/81 [==============================] - 2s 28ms/step - loss: 3060.3507
Epoch 5/200
81/81 [==============================] - 2s 27ms/step - loss: 2362.2936
Epoch 6/200
81/81 [==============================] - 2s 27ms/step - loss: 1767.2917
Epoch 7/200
81/81 [==============================] - 2s 27ms/step - loss: 1416.9346
Epoch 8/200
81/81 [==============================] - 3s 33ms/step - loss: 1041.5027
Epoch 9/200
81/81 [==============================] - 2s 28ms/step - loss: 795.6069
Epoch 10/200
81/81 [==============================] - 2s 27ms/step - loss: 594.2081
Epoch 11/200
81/81 [==============================] - 2s 27ms/step - loss: 486.9300
Epoch 12/200
81/81 [==============================] - 2s 28ms/step - loss: 

In [123]:
#Make predictions for futre prices
prediction = model.predict_generator(test_generator)

#Test model predictions for accuracy through plottting
close_train = close_train.reshape((-1))
close_test = close_test.reshape((-1))
prediction = prediction.reshape((-1))

#PLot for training prices
trace1 = go.Scatter(
    x = date_train,
    y = close_train,
    mode = 'lines',
    name = 'Data'
)
# PLotting our predicted prices
trace2 = go.Scatter(
    x = date_test,
    y = prediction,
    mode = 'lines',
    name = 'Prediction'
)
#plotting the actual prices for the period of time that was predicted
trace3 = go.Scatter(
    x = date_test,
    y = close_test,
    mode='lines',
    name = 'Ground Truth'
)
#Plot features and labels
layout = go.Layout(
    title = "VNQ",
    xaxis = {'title' : "Date"},
    yaxis = {'title' : "Close"}
)
#Show plot
fig = go.Figure(data=[trace1, trace2, trace3], layout=layout)
fig.show()

C:\Users\lwbak\.conda\lib\site-packages\tensorflow\python\keras\engine\training.py:1905: UserWarning:

`Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.



In [124]:
#Predicting future prices

close_data = close_data.reshape((-1))

def predict(num_prediction, model):
    prediction_list = close_data[-look_back:]
    
    for _ in range(num_prediction):
        x = prediction_list[-look_back:]
        x = x.reshape((1, look_back, 1))
        out = model.predict(x)[0][0]
        prediction_list = np.append(prediction_list, out)
    prediction_list = prediction_list[look_back-1:]
        
    return prediction_list
    
def predict_dates(num_prediction):
    last_date = ETF['Date'].values[-1]
    prediction_dates = pd.date_range(last_date, periods=num_prediction+1).tolist()
    return prediction_dates

num_prediction = 90
forecast = predict(num_prediction, model)
forecast_dates = predict_dates(num_prediction)

In [125]:
forecast

array([84.31      , 81.11442566, 80.12725067, 80.20650482, 80.16213989,
       80.22246552, 79.90106964, 79.89716339, 79.06156921, 79.38271332,
       79.07112885, 79.10837555, 79.03273773, 79.43930054, 79.57221222,
       79.58246613, 79.77222443, 79.97045898, 80.18760681, 80.29122162,
       80.10340118, 80.41426849, 80.18871307, 80.28702545, 80.21190643,
       79.86390686, 79.78994751, 79.89667511, 79.57244873, 79.63568115,
       79.65890503, 79.46342468, 79.53270721, 79.29608154, 79.36599731,
       78.83183289, 79.14975739, 79.04854584, 79.19841766, 79.57109833,
       79.5786972 , 80.0428772 , 79.81655121, 80.17805481, 80.35134888,
       80.86134338, 80.53701782, 80.89611053, 80.73466492, 81.11373901,
       80.74985504, 80.79164124, 80.75222778, 80.68605042, 80.70662689,
       80.63336182, 80.79695129, 80.40029144, 80.69812012, 80.70495605,
       80.55071259, 80.83032227, 80.69680786, 80.61231995, 80.50565338,
       80.35362244, 80.57170105, 80.30702209, 80.69398499, 80.42

In [126]:
#making a dataframe
predicted_prices = pd.DataFrame({'Date': forecast_dates, 'VNQ_close_price': list(forecast)}, columns=['Date', 'VNQ_close_price'])
predicted_prices.head()

,Date,VNQ_close_price
0,2021-01-13 00:00:00-05:00,84.310000
1,2021-01-14 00:00:00-05:00,81.114426
2,2021-01-15 00:00:00-05:00,80.127251
3,2021-01-16 00:00:00-05:00,80.206505
4,2021-01-17 00:00:00-05:00,80.162140


In [127]:
#export to excel
predicted_prices.to_csv(r'Resources/VNQ_forecast.csv')

In [128]:
# Combine all forecasted prices into a single DF
LQDpath = Path("Resources/LQD_forecast.csv")
LQD = pd.read_csv(LQDpath)
#LQD.set_index(LQD['Date'], inplace=True)

PZApath = Path("Resources/PZA_forecast.csv")
PZA = pd.read_csv(PZApath)
PZA = PZA['PZA_close_price']
#PZA.set_index(PZA['Date'], inplace=True)

USOpath = Path("Resources/USO_forecast.csv")
USO = pd.read_csv(USOpath)
USO = USO['USO_close_price']
#USO.set_index(USO['Date'], inplace=True)

TLTpath = Path("Resources/TLT_forecast.csv")
TLT = pd.read_csv(TLTpath)
TLT = TLT['TLT_close_price']
#TLT.set_index(TLT['Date'], inplace=True)

QQQpath = Path("Resources/QQQ_forecast.csv")
QQQ = pd.read_csv(QQQpath)
QQQ = QQQ['QQQ_close_price']
#QQQ.set_index(QQQ['Date'], inplace=True)

VMBSpath = Path("Resources/VMBS_forecast.csv")
VMBS = pd.read_csv(VMBSpath)
VMBS = VMBS['VMBS_close_price']
#VMBS.set_index(VMBS['Date'], inplace=True)

VNQpath = Path("Resources/VNQ_forecast.csv")
VNQ = pd.read_csv(VNQpath)
VNQ = VNQ['VNQ_close_price']

QQQ.head()

0    316.000000
1    303.799408
2    302.885742
3    302.805603
4    302.869537
Name: QQQ_close_price, dtype: float64

In [129]:
#Concatenate and clean the data
forecast_prices = pd.concat([LQD, PZA, USO, TLT, QQQ, VMBS, VNQ], axis='columns', join='inner')
forecast_prices = forecast_prices.drop(columns=["Unnamed: 0"])
forecast_prices.head()

,Date,LQD_close_price,PZA_close_price,USO_close_price,TLT_close_price,QQQ_close_price,VMBS_close_price,VNQ_close_price
0,2021-01-13 00:00:00-05:00,136.520000,27.090000,35.760000,152.360000,316.000000,54.010000,84.310000
1,2021-01-14 00:00:00-05:00,137.277298,27.027870,35.348991,154.565033,303.799408,54.105686,81.114426
2,2021-01-15 00:00:00-05:00,138.050400,26.947447,35.693897,155.340942,302.885742,54.160480,80.127251
3,2021-01-16 00:00:00-05:00,138.771729,26.877699,35.377987,157.228912,302.805603,54.233402,80.206505
4,2021-01-17 00:00:00-05:00,139.553513,26.820807,35.770760,157.639771,302.869537,54.292553,80.162140


In [130]:
#EXPORT EXCEL
forecast_prices.to_csv(r'Resources/forecast_prices.csv')